## **แบบฝึกหัดขั้นสูง: Multi-Function Calling และการจัดการข้อมูลด้วย Pydantic**


---

### **ข้อที่ 1: ระบบวางแผนการเดินทางอัจฉริยะ**

**สถานการณ์:** ผู้ใช้ต้องการวางแผนการเดินทางจากโตเกียว (Tokyo) ไปยังปารีส (Paris) และต้องการข้อมูลเปรียบเทียบ ทั้งเรื่องเวลาท้องถิ่นของทั้งสองเมือง, สภาพอากาศปัจจุบัน, และเมืองหลวงของประเทศนั้นๆ เพื่อยืนยันข้อมูล

**งานของคุณ:**
1.  เขียนโค้ดที่รับคำสั่งจากผู้ใช้ เช่น `"ฉันจะเดินทางจากโตเกียวไปปารีส ช่วยบอกเวลาท้องถิ่นและสภาพอากาศของทั้งสองเมืองหน่อย แล้วเมืองหลวงของญี่ปุ่นกับฝรั่งเศสคือที่ไหนนะ?"`
2.  คุณจะต้องสร้าง Mock Function ใหม่ชื่อ `get_weather(city: str)` ซึ่งจะคืนค่าเป็นข้อมูลสภาพอากาศจำลอง (เช่น `{"temperature": "15°C", "condition": "Cloudy"}`)
3.  เรียกใช้ฟังก์ชัน `get_current_time`, `get_capital_city`, และ `get_weather` สำหรับเมืองทั้งสอง
4.  ** (ส่วนที่ท้าทาย)** ออกแบบ Pydantic Model ชื่อ `TripPlannerResponse` เพื่อรวบรวมข้อมูลทั้งหมด โดยมีการแบ่งโครงสร้างข้อมูลของแต่ละเมืองอย่างชัดเจน

**Pydantic Model ที่คาดหวัง:**
```python
from pydantic import BaseModel
from typing import Dict

class CityInfo(BaseModel):
    local_time: str
    weather: Dict[str, str] # e.g., {"temperature": "15°C", "condition": "Cloudy"}
    capital_of: str

class TripPlannerResponse(BaseModel):
    origin: CityInfo
    destination: CityInfo
    trip_summary: str # สร้างข้อความสรุปสั้นๆ จากข้อมูลทั้งหมด
```
**คำใบ้:** โมเดล LLM อาจจะต้องเรียกฟังก์ชันเดียวกัน (เช่น `get_current_time`) แต่ด้วยพารามิเตอร์ที่ต่างกัน (คนละเมือง) ในคำสั่งเดียว คุณต้องจัดการผลลัพธ์เหล่านี้ให้ถูกต้องและจับคู่กับเมืองที่ถูกต้องใน Pydantic Model

In [ ]:
import IPython
import sys

def clean_notebook():
    IPython.display.clear_output(wait=True)
    print("Notebook cleaned.")

# Install the required packages
!pip install openai python-dotenv

# Clean up the notebook output
clean_notebook()

In [27]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [28]:
import os
from openai import OpenAI

# Initialize the OpenAI client with the API key
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
model_name = "gpt-4.1"

In [ ]:
import json
from openai import OpenAI
from pydantic import BaseModel
import os
import random
import datetime
import pytz
from dotenv import load_dotenv

client = OpenAI()



None


In [ ]:
from datetime import datetime
import pytz

import requests

# Real-time function to get current time of a city
def get_current_time(city: str) -> str:
    # ตั้งค่าเขตเวลาของแต่ละเมือง
    timezones = {
        "Tokyo": "Asia/Tokyo",
        "Paris": "Europe/Paris",
    }
    
    if city not in timezones:
        return "City not found"
    
    timezone = pytz.timezone(timezones[city])
    local_time = datetime.now(timezone)
    return local_time.strftime("%Y-%m-%d %H:%M:%S")



# สมมติว่าเรามีไฟล์ JSON ที่เก็บข้อมูลสภาพอากาศ
weather_data = {
    "Tokyo": {"temperature": "15°C", "condition": "Cloudy"},
    "Paris": {"temperature": "18°C", "condition": "Sunny"}
}

def get_weather(city: str):
    if city in weather_data:
        return weather_data[city]
    else:
        return "Weather data not found"




# Real-time function to get capital city
def get_capital_city(country: str) -> str:
    url = f"https://restcountries.com/v3.1/name/{country}"
    response = requests.get(url)
    
    if response.status_code == 200:
        country_data = response.json()
        return country_data[0]["capital"][0] if "capital" in country_data[0] else "Capital not found"
    else:
        return "Capital not found"



2025-08-08 03:41:36
{'temperature': '15°C', 'condition': 'Cloudy'}


In [77]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather of a specified city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city, e.g., Tokyo, Paris"
                    }
                },
                "required": ["city"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "Get the current local time for a given city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "City name, e.g., Tokyo, Paris"
                    }
                },
                "required": ["city"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_capital_city",
            "description": "Get the capital city of a given country.",
            "parameters": {
                "type": "object",
                "properties": {
                    "country": {
                        "type": "string",
                        "description": "Country name, e.g., Japan, France"
                    }
                },
                "required": ["country"]
            }
        }
    }
]



In [127]:
import json

def execute_multi_function_call(user_query, tools):
    """A helper function to execute the full multi-function calling flow."""
    messages = [{"role": "user", "content": user_query}]
    
    # First API call to the model
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    response_message = response.choices[0].message
    messages.append(response_message)  # Append the model's response
    
    # Check if the model wants to call one or more functions
    if hasattr(response_message, "tool_calls") and response_message.tool_calls:
        available_functions = {
            "get_current_time": get_current_time,
            "get_capital_city": get_capital_city,
            "get_weather": get_weather,
        }

        # Loop through each tool call requested by the model
        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions.get(function_name)
            
            if function_to_call:
                function_args = json.loads(tool_call.function.arguments)
                try:
                    function_response = function_to_call(**function_args)
                    # Ensure the response is converted to a string
                    function_response_str = json.dumps(function_response) if isinstance(function_response, dict) else str(function_response)
                except TypeError as e:
                    # In case of any argument mismatch, log the error
                    function_response_str = f"Error: {e}"

                # Send the function's result back to the model
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": function_response_str  # Ensure content is a string
                })

        # Second API call to get a natural language response
        second_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages
        )
        return second_response.choices[0].message.content
    else:
        return response_message.content



In [128]:
user_query = "ต้องการวางแผนการเดินทางจากโตเกียว (Tokyo) ไปยังปารีส (Paris) และต้องการข้อมูลเปรียบเทียบ ทั้งเรื่องเวลาท้องถิ่นของทั้งสองเมือง, สภาพอากาศปัจจุบัน, และเมืองหลวงของประเทศนั้นๆ เพื่อยืนยันข้อมูล"
final_response = execute_multi_function_call(user_query, tools)
print(final_response)


ข้อมูลเปรียบเทียบสำหรับการเดินทางจากโตเกียวไปยังปารีส:

1. **เวลาท้องถิ่น:**
   - โตเกียว: วันที่ 8 สิงหาคม 2025 เวลา 06:45 น.
   - ปารีส: วันที่ 7 สิงหาคม 2025 เวลา 23:45 น.

2. **สภาพอากาศปัจจุบัน:**
   - โตเกียว: อุณหภูมิ 15°C, มีเมฆมาก
   - ปารีส: อุณหภูมิ 18°C, แดดส่อง

3. **เมืองหลวง:**
   - ประเทศญี่ปุ่น: โตเกียว
   - ประเทศฝรั่งเศส: ปารีส

ข้อมูลทั้งหมดนี้สามารถช่วยให้คุณวางแผนการเดินทางได้อย่างมีประสิทธิภาพและมั่นใจมากขึ้นค่ะ!


In [ ]:
from openai import OpenAI
from pydantic import BaseModel
from typing import List

client = OpenAI()

class WeatherInfo(BaseModel):
    temperature: str
    condition: str

class CityInfo(BaseModel):
    local_time: str
    weather: WeatherInfo
    capital_of: str

class TripPlannerResponse(BaseModel):
    origin: CityInfo
    destination: CityInfo
    trip_summary: str

messages = [
    {"role": "system", "content": "Extract the information into the TripPlannerResponse format"},
    {"role": "user", "content": str(final_response)}
]

response = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=messages,
    response_format=TripPlannerResponse
)

parsed = response.choices[0].message.parsed
json_result = json.dumps(parsed.model_dump(), ensure_ascii=False, indent=4)
print(json_result)
print(final_response)


{
    "origin": {
        "local_time": "8 สิงหาคม 2025 เวลา 06:45 น.",
        "weather": {
            "temperature": "15°C",
            "condition": "มีเมฆมาก"
        },
        "capital_of": "ประเทศญี่ปุ่น"
    },
    "destination": {
        "local_time": "7 สิงหาคม 2025 เวลา 23:45 น.",
        "weather": {
            "temperature": "18°C",
            "condition": "แดดส่อง"
        },
        "capital_of": "ประเทศฝรั่งเศส"
    },
    "trip_summary": "ข้อมูลทั้งหมดนี้สามารถช่วยให้คุณวางแผนการเดินทางได้อย่างมีประสิทธิภาพและมั่นใจมากขึ้นค่ะ!"
}
ข้อมูลเปรียบเทียบสำหรับการเดินทางจากโตเกียวไปยังปารีส:

1. **เวลาท้องถิ่น:**
   - โตเกียว: วันที่ 8 สิงหาคม 2025 เวลา 06:45 น.
   - ปารีส: วันที่ 7 สิงหาคม 2025 เวลา 23:45 น.

2. **สภาพอากาศปัจจุบัน:**
   - โตเกียว: อุณหภูมิ 15°C, มีเมฆมาก
   - ปารีส: อุณหภูมิ 18°C, แดดส่อง

3. **เมืองหลวง:**
   - ประเทศญี่ปุ่น: โตเกียว
   - ประเทศฝรั่งเศส: ปารีส

ข้อมูลทั้งหมดนี้สามารถช่วยให้คุณวางแผนการเดินทางได้อย่างมีประสิทธิภาพและมั่นใจมากขึ้นค่ะ!


In [ ]:
# # ตัวอย่างการใช้ OpenAI API (สมมุติว่า client ถูกตั้งค่าไว้แล้ว)
# response = client.chat.completions.create(
#     model="gpt-4o",  # ชื่อโมเดลที่ใช้
#     messages=[{
#         "role": "system", "content": "You are a helpful assistant."},
#         {
#         "role": "user", "content": "ต้องการวางแผนการเดินทางจากโตเกียว (Tokyo) ไปยังปารีส (Paris) และต้องการข้อมูลเปรียบเทียบ ทั้งเรื่องเวลาท้องถิ่นของทั้งสองเมือง, สภาพอากาศปัจจุบัน, และเมืองหลวงของประเทศนั้นๆ เพื่อยืนยันข้อมูล"
#     }],
#     tools=tools,  # ใช้ตัวแปร tools ที่แยกออกมา
#     tool_choice="auto"  # เลือกเครื่องมือที่จะใช้โดยอัตโนมัติ
# )

### **ข้อที่ 2: ระบบสรุปข้อมูลเหตุการณ์สำคัญ**

**สถานการณ์:** นักข่าวต้องการข้อมูลสรุปเกี่ยวกับเหตุการณ์ 'การประชุมสุดยอด AI' โดยต้องการทราบว่าจัดขึ้นที่ประเทศใด (สมมติว่าจัดขึ้นที่ฝรั่งเศส), เมืองหลวงของประเทศนั้น, เวลาท้องถิ่น ณ ตอนนั้น, และข่าวล่าสุดเกี่ยวกับ 'AI' และ 'เศรษฐศาสตร์ (economics)' เพื่อหาความเชื่อมโยง

**งานของคุณ:**
1.  เขียนโค้ดที่รับคำสั่งจากผู้ใช้ เช่น `"การประชุมสุดยอด AI จัดขึ้นที่ไหน ช่วยบอกเมืองหลวง เวลาปัจจุบัน และสรุปข่าวล่าสุดเกี่ยวกับ AI และเศรษฐศาสตร์ให้ที"`
2.  LLM ควร (infer) ได้ว่าต้องหาเมืองหลวงของฝรั่งเศส และเรียกใช้ `get_capital_city`, `get_current_time`, และ `get_latest_news` (สองครั้งสำหรับสองหัวข้อ)
3.  **(ส่วนที่ท้าทาย)** ออกแบบ Pydantic Model ที่สามารถรวบรวม 'หัวข้อข่าว' หลายๆ หัวข้อได้แบบไดนามิก และสร้างข้อความสรุปเชิงวิเคราะห์สั้นๆ จากข้อมูลทั้งหมด

**Pydantic Model ที่คาดหวัง:**
```python
from pydantic import BaseModel, Field
from typing import Dict

class EventBriefing(BaseModel):
    event_name: str = "AI Summit"
    location_country: str
    location_capital: str
    local_time: str
    news_briefings: Dict[str, str] = Field(..., description="A dictionary mapping news topics to their summaries")
    analytic_summary: str # ข้อความสรุปที่สร้างขึ้นเอง เช่น 'The AI Summit in Paris is happening amidst news of powerful AI models, which may impact the economy.'
```
**คำใบ้:** การที่ LLM ต้องเรียก `get_latest_news` สองครั้งสำหรับสองหัวข้อที่แตกต่างกัน เป็นการทดสอบความสามารถในการจัดการ Tool Calls ที่ซ้ำซ้อนแต่มีพารามิเตอร์ต่างกัน คุณต้องมี Logic ในการรวบรวมผลลัพธ์จาก Tool Calls เหล่านี้ลงใน Dictionary ของ Pydantic

In [147]:
from datetime import datetime
import pytz

import requests

# Real-time function to get current time of a city
def get_current_time(city: str) -> str:
    # ตั้งค่าเขตเวลาของแต่ละเมือง
    timezones = {
        "Tokyo": "Asia/Tokyo",
        "Paris": "Europe/Paris",
    }
    
    if city not in timezones:
        return "City not found"
    
    timezone = pytz.timezone(timezones[city])
    local_time = datetime.now(timezone)
    return local_time.strftime("%Y-%m-%d %H:%M:%S")



# สมมติว่าเรามีไฟล์ JSON ที่เก็บข้อมูลสภาพอากาศ
weather_data = {
    "Tokyo": {"temperature": "15°C", "condition": "Cloudy"},
    "Paris": {"temperature": "18°C", "condition": "Sunny"}
}

def get_weather(city: str):
    if city in weather_data:
        return weather_data[city]
    else:
        return "Weather data not found"




# Real-time function to get capital city
def get_capital_city(country: str) -> str:
    url = f"https://restcountries.com/v3.1/name/{country}"
    response = requests.get(url)
    
    if response.status_code == 200:
        country_data = response.json()
        return country_data[0]["capital"][0] if "capital" in country_data[0] else "Capital not found"
    else:
        return "Capital not found"
    
def get_latest_news(topic: str) -> str:
    news = {
        "AI": "AI advancements continue to make waves in various industries. Recently, a new AI model has been unveiled that can simulate human-like conversation and solve complex problems, significantly impacting fields such as healthcare and customer service.",
        "economics": "The global economy is undergoing significant changes, with inflation rates on the rise. Experts are predicting that countries will need to implement tighter monetary policies to stabilize their economies, especially amidst ongoing challenges like supply chain disruptions and geopolitical tensions."
    }
    
    return news.get(topic, "No news available for this topic")


def generate_event_briefing(country: str, city: str):
    # ดึงข้อมูลเกี่ยวกับเวลาและเมืองหลวง
    local_time = get_current_time(city)
    capital_city = get_capital_city(country)
    
    # ดึงข้อมูลข่าวล่าสุดจากสองหัวข้อ
    ai_news = get_latest_news("AI")
    econ_news = get_latest_news("economics")
    
    # สร้างข้อความสรุป
    analytic_summary = f"The AI Summit in {capital_city} is happening amidst discussions on {ai_news} and the latest economic trends such as {econ_news}."
    
    # สร้าง dictionary ผลลัพธ์
    event_briefing = {
        "event_name": "AI Summit",
        "location_country": country,
        "location_capital": capital_city,
        "local_time": local_time,
        "news_briefings": {
            "AI": ai_news,
            "economics": econ_news
        },
        "analytic_summary": analytic_summary
    }
    
    return event_briefing

# ตัวอย่างการใช้งาน
event_briefing = generate_event_briefing("France", "Paris")
print(event_briefing)




{'event_name': 'AI Summit', 'location_country': 'France', 'location_capital': 'Paris', 'local_time': '2025-08-08 01:50:35', 'news_briefings': {'AI': 'AI advancements continue to make waves in various industries. Recently, a new AI model has been unveiled that can simulate human-like conversation and solve complex problems, significantly impacting fields such as healthcare and customer service.', 'economics': 'The global economy is undergoing significant changes, with inflation rates on the rise. Experts are predicting that countries will need to implement tighter monetary policies to stabilize their economies, especially amidst ongoing challenges like supply chain disruptions and geopolitical tensions.'}, 'analytic_summary': 'The AI Summit in Paris is happening amidst discussions on AI advancements continue to make waves in various industries. Recently, a new AI model has been unveiled that can simulate human-like conversation and solve complex problems, significantly impacting fields 

In [148]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather of a specified city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city, e.g., Tokyo, Paris"
                    }
                },
                "required": ["city"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "Get the current local time for a given city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "City name, e.g., Tokyo, Paris"
                    }
                },
                "required": ["city"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_capital_city",
            "description": "Get the capital city of a given country.",
            "parameters": {
                "type": "object",
                "properties": {
                    "country": {
                        "type": "string",
                        "description": "Country name, e.g., Japan, France"
                    }
                },
                "required": ["country"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_latest_news",
            "description": "Get the latest news for a specified topic (e.g., AI, economics).",
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {
                        "type": "string",
                        "description": "The topic for which to fetch the latest news, e.g., AI, economics"
                    }
                },
                "required": ["topic"]
            }
        }
    }
]


In [149]:
import json

def execute_multi_function_call(user_query, tools):
    """A helper function to execute the full multi-function calling flow."""
    messages = [{"role": "user", "content": user_query}]
    
    # First API call to the model
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    response_message = response.choices[0].message
    messages.append(response_message)  # Append the model's response
    
    # Check if the model wants to call one or more functions
    if hasattr(response_message, "tool_calls") and response_message.tool_calls:
        available_functions = {
            "get_current_time": get_current_time,
            "get_capital_city": get_capital_city,
            "get_weather": get_weather,
            "get_latest_news": get_latest_news  # Added new function for getting latest news
        }

        # Loop through each tool call requested by the model
        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions.get(function_name)
            
            if function_to_call:
                function_args = json.loads(tool_call.function.arguments)
                try:
                    function_response = function_to_call(**function_args)
                    # Ensure the response is converted to a string
                    function_response_str = json.dumps(function_response) if isinstance(function_response, dict) else str(function_response)
                except TypeError as e:
                    # In case of any argument mismatch, log the error
                    function_response_str = f"Error: {e}"

                # Send the function's result back to the model
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": function_response_str  # Ensure content is a string
                })

        # Second API call to get a natural language response
        second_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages
        )
        return second_response.choices[0].message.content
    else:
        return response_message.content


In [ ]:
user_query = "นักข่าวต้องการข้อมูลสรุปเกี่ยวกับเหตุการณ์ 'การประชุมสุดยอด AI' โดยต้องการทราบว่าจัดขึ้นที่ประเทศใด (สมมติว่าจัดขึ้นที่ฝรั่งเศส), เมืองหลวงของประเทศนั้น, เวลาท้องถิ่น ณ ตอนนั้น, และข่าวล่าสุดเกี่ยวกับ 'AI' และ 'เศรษฐศาสตร์ (economics)' เพื่อหาความเชื่อมโยง"
report = execute_multi_function_call(user_query, tools)
print(report)


การประชุมสุดยอด AI ได้จัดขึ้นที่ประเทศฝรั่งเศส โดยเมืองหลวงของฝรั่งเศสคือ "ปารีส" เวลาท้องถิ่นในขณะนั้นคือวันที่ 8 สิงหาคม 2025 เวลา 01:50 น.

สำหรับข่าวล่าสุดเกี่ยวกับ AI มีการเปิดตัวโมเดล AI ใหม่ที่สามารถสนทนาเหมือนมนุษย์และแก้ปัญหาที่ซับซ้อนได้ ซึ่งมีผลกระทบอย่างมากต่อภาคส่วนต่างๆ เช่น การแพทย์และการบริการลูกค้า

ทางด้านเศรษฐศาสตร์ เศรษฐกิจโลกกำลังเผชิญกับการเปลี่ยนแปลงอย่างมีนัยสำคัญ อัตราเงินเฟ้อที่เพิ่มขึ้นทำให้ผู้เชี่ยวชาญคาดการณ์ว่าประเทศต่างๆ จะต้องดำเนินนโยบายการเงินที่เข้มงวดขึ้นเพื่อรักษาเสถียรภาพของเศรษฐกิจ โดยเฉพาะภายใต้อุปสรรคที่กำลังเผชิญอยู่ เช่น การหยุดชะงักของห่วงโซ่อุปทานและความตึงเครียดทางภูมิรัฐศาสตร์

สามารถสังเกตเห็นความเชื่อมโยงได้ว่า การพัฒนาในด้าน AI ไม่เพียงแต่จะส่งผลดีต่ออุตสาหกรรมและการบริการต่างๆ แต่ยังมีศักยภาพที่จะช่วยในการคาดการณ์และวิเคราะห์สภาวะเศรษฐกิจ เพื่อช่วยให้สามารถปรับตัวได้อย่างมีประสิทธิภาพในสภาพเศรษฐกิจที่แปรปรวน


In [162]:
from openai import OpenAI
from pydantic import BaseModel
from pydantic import BaseModel, Field
from typing import Dict , Optional

client = OpenAI()



class NewsInfo(BaseModel):
    summary: str  # สรุปข่าว
    source: str   # แหล่งที่มาของข่าว

class EventBriefing(BaseModel):
    event_name: str = "AI Summit"
    location_country: str
    location_capital: str
    local_time: str
    # ใช้ NewsInfo แทน Dict
    news_briefings: Optional[dict[str, NewsInfo]] = Field(None, description="A dictionary mapping news topics to NewsInfo objects")
    analytic_summary: str  # Analytical summary of the event
    
    
messages = [
    {"role": "system", "content": "Extract the information into the TripPlannerResponse format"},
    {"role": "user", "content": str(final_response)}
]

response = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=messages,
    response_format=EventBriefing
)

parsed = response.choices[0].message.parsed
json_result = json.dumps(parsed.model_dump(), ensure_ascii=False, indent=4)
print(json_result)
print(final_response)


{
    "event_name": "การประชุมสุดยอด AI",
    "location_country": "ฝรั่งเศส",
    "location_capital": "ปารีส",
    "local_time": "2025-08-08T01:50:00",
    "news_briefings": {
        "AI": {
            "summary": "มีการเปิดตัวโมเดล AI ใหม่ที่สามารถสนทนาเหมือนมนุษย์และแก้ปัญหาที่ซับซ้อนได้ ซึ่งส่งผลกระทบอย่างมากต่อการแพทย์และการบริการลูกค้า",
            "source": "การประชุมสุดยอด AI"
        },
        "เศรษฐศาสตร์": {
            "summary": "เศรษฐกิจโลกกำลังเผชิญกับการเปลี่ยนแปลง มีอัตราเงินเฟ้อที่เพิ่มขึ้นและประเทศต้องดำเนินนโยบายการเงินที่เข้มงวดขึ้นเพื่อรับมือกับการหยุดชะงักของห่วงโซ่อุปทานและความตึงเครียดทางภูมิรัฐศาสตร์",
            "source": "การประชุมสุดยอด AI"
        }
    },
    "analytic_summary": "การพัฒนาในด้าน AI มีศักยภาพในการช่วยคาดการณ์และวิเคราะห์สภาวะเศรษฐกิจ ช่วยองค์กรต่างๆ ในการปรับตัวได้ในสภาพเศรษฐกิจที่แปรปรวน"
}
การประชุมสุดยอด AI ได้จัดขึ้นที่ประเทศฝรั่งเศส โดยเมืองหลวงของฝรั่งเศสคือ "ปารีส" เวลาท้องถิ่นในขณะนั้นคือวันที่ 8 สิงหาคม 2025 เวลา 01:50 น.

สำหรั